In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import aiospotify
from aiospotify import Spotify
import spotipy

import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
ACCESS_TOKEN = "BQDTqYJ4aHQzLUuyA3NYAnC1d4q1s_afF5SW5opx1fm4qfi1eoIxDriRDh7A6zDORSlftsEhPS0dSlsMnpuFrFVMLX-vhWt5H73jUfHg2EY1pGJQdpGtKRJnRh1ePYqshezELyrKm7OMSpQMhAsgeJpSj6TwXX17p1MPyIvAnl8"
spotify = Spotify(auth=ACCESS_TOKEN)

In [5]:
result = await spotify.current_user_saved_tracks()
track_list = result.items

In [5]:
track_list[0]["track"].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [32]:
# Extract track IDs so we can fetch additional info
track_id_list = [track.id for track in track_list]
# Get track audio features
track_features_list = await spotify.audio_features(track_id_list)
# Get track audio analysis
track_analysis_list = [await spotify.audio_analysis(track_id) for track_id in track_id_list]

AttributeError: 'dict' object has no attribute 'id'

In [38]:
def get_features(*, track=None, audio_features=None, audio_analysis=None):
    features = {}
    if track:
        keys = ("popularity",)
        features.update({key: track[key] for key in keys})
    if audio_features:
        keys = ("danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo")
        features.update({key: audio_features[key] for key in keys})
    if audio_analysis:
        audio_analysis_track_features = audio_analysis["track"]
        keys = ("mode",)
#         features.update({key: audio_analysis_track_features[key] for key in keys})
    return features

In [39]:
features_list = [get_features(track=track_list[idx], audio_features=track_features_list[idx], audio_analysis=track_analysis_list[idx]) for idx in range(len(track_id_list))]

In [40]:
features_df = pd.DataFrame(features_list)
# normalized_features_df = (features_df - features_df.mean()) / features_df.std()
normalized_features_df = (features_df - features_df.min()) / (features_df.max() - features_df.min())

In [41]:
import umap
embedder = umap.UMAP()
embedding = embedder.fit_transform(np.array(normalized_features_df))

In [57]:
import umap.plot
umap.plot.output_notebook()
p = umap.plot.interactive(embedder, values=normalized_features_df["energy"], hover_data=normalized_features_df, theme="viridis")
umap.plot.show(p)

Loading BokehJS ...

In [ ]:
g = nx.Graph()